In [7]:
import boto3
import duckdb
import zipfile
import io

In [8]:
s3 = boto3.client('s3')

response = s3.get_object(
    Bucket='public-transit-data-raw',
    Key='2025-03-30/sp_trans_data_11_13_41.zip'
)['Body'].read()



In [9]:
with zipfile.ZipFile(io.BytesIO(response), "r") as file:
    print(file.namelist())

['agency.txt', 'calendar.txt', 'fare_attributes.txt', 'fare_rules.txt', 'frequencies.txt', 'routes.txt', 'shapes.txt', 'stops.txt', 'stop_times.txt', 'trips.txt']


In [10]:
FILE_NAME_MAPPING = {
    "agency.txt": "agencias",
    "calendar.txt": "calendario_servico",
    "fare_attributes.txt": "tarifas_atributos",
    "fare_rules.txt": "regras_tarifarias",
    "frequencies.txt": "frequencias_veiculos",
    "routes.txt": "rotas",
    "shapes.txt": "trajetos_geograficos",
    "stops.txt": "pontos_parada",
    "stop_times.txt": "horarios_paradas",
    "trips.txt": "viagens"
}

In [12]:
import tempfile
import os

from os.path import abspath

conn = duckdb.connect()

with zipfile.ZipFile(io.BytesIO(response), "r") as zip_file:
        for file_name in zip_file.namelist():
            if not file_name.endswith(".txt"):
                continue

            # Cria caminho temporário para salvar o .txt
            tmp_dir = tempfile.gettempdir()
            txt_path = os.path.join(tmp_dir, file_name)

            with zip_file.open(file_name) as txt_file:
                with open(txt_path, "wb") as f_out:
                    f_out.write(txt_file.read())
            
            sql_path = abspath(f"table_schema/{file_name.replace('.txt', '.sql')}")

            with open(sql_path, "r") as sql_file:
                query = sql_file.read().format(txt_path=txt_path, extraction_date="2025-03-30")

            df = conn.execute(query).fetchdf()
            print(df)

            new_file_name = FILE_NAME_MAPPING.get(file_name, file_name.replace(".txt", ""))
            break


   id_agencia nome_agencia                               url_agencia  \
0           1      SPTRANS  http://www.sptrans.com.br/?versao=260325   

  fuso_horario_agencia idioma_agencia data_extracao  
0    America/Sao_Paulo             pt    2025-03-30  
